<a href="https://colab.research.google.com/github/ch00226855/CMP414765Fall2022/blob/main/Week03_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3
# Simple Linear Regression and the Normal Equation

Last week, we used the `LinearRegression` tool from `sklearn` to find a mathematical expression to describe the relationship between two variables. This week, we are going to study how this method works.

**Reading**: Textbook, Chapter 4

## I. Simple Linear regression: Sales Prediction

To put things into context, let's look at a dataset that contains the sales revenue and the advertising budgets of a company in 200 different markets.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
url = "https://www.statlearning.com/s/Advertising.csv"
advertising = pd.read_csv(url, index_col=0)
advertising.head()

In [ ]:
advertising.info()

In [ ]:
advertising.describe()

In [ ]:
fig = advertising.hist(figsize=(10, 10))

In [ ]:
# plot TV vs. sales
plt.plot(advertising['TV'], advertising['sales'], 'g.')
plt.title("TV vs. sales")
plt.xlabel("TV budget")
plt.ylabel("Sales Revenue")
plt.savefig("TVvsSales.png")
plt.show() # this is needed in other Python environments

In [ ]:
# Exercise:
# plot radio vs. sales



What can we say with these scatter plots?
- There is a strong positive correlation between TV and sales
- There is a positive correlation between radio and sales
- I do not observe a strong correlation between newspaper and sales. 

In [ ]:
# The outcome from the describe() method suggests there are outliers.

# Use a filter to extract the record with 0.7 TV value
advertising[advertising["TV"] == 0.7] # the condition in [] creates a list of boolean indices


In [ ]:
# Sort the records in the ascending order of TV
advertising.sort_values(by="TV", ascending=False, inplace=True)
advertising = advertising.sort_values(by="TV", ascending=False)

In [ ]:
advertising.head()

### Correlation Coefficient: Numerical Measure of Correlations
The **correlation coefficient** is a numerical measurement of **linear correlation** between two variables.
- The value of correlation coefficient always lies in [-1, 1].
- If there is a strong positive correlation, then the coefficient is close to 1.
- If there is a strong negative correlation, then the coefficient is close to -1.
- If there is a very weak correlation, then the coefficient is close to 0.
- However, a near-zero coeffient may be caused by non-linear correlations.
![](https://upload.wikimedia.org/wikipedia/commons/d/d4/Correlation_examples2.svg)

In [ ]:
# Calculate pair-wise correlation coefficients
advertising.corr()

For simplicity, we will only use `TV` as a predictor of `sales`.

In [ ]:
data = advertising.loc[:, ['TV', 'sales']] 
# The loc expression require both row indices and column names
data.head()

In [ ]:
# Use the loc expression to find the sale value for row 100
advertising.loc[100, 'sales']

# Use the loc expression to find the sales value for row 50 to 60
advertising = advertising.reset_index(drop=True)
advertising.loc[[50, 52, 69], 'sales']

## Simple Linear Regression: Model Representation

In order to describe the model mathematically, we need to introduce a few notations:
- The input feature `TV` is represented as variable $X$.
- The output/response feature `sales` is represented as variable $Y$.
- Each instance of data is represented as $(x^{(i)}, y^{(i)})$, where $i$ is the row index, $x^{(i)}$ is the value corresponding to $X$, and $y^{(i)}$ is the value corresponding to $Y$. For example, $(x^{(0)}, y^{(0)}) = (296.4, 23.8)$.
- Let $N$ represent the total number of data. In this case, N = 200.

The **simple linear regression** model assumes that the relationship between $X$ and $Y$ is
$$Y \approx f(X) = \beta_0 + \beta_1 X.$$

- $\beta_0$ and $\beta_1$ are called **model parameters**. For simple linear regression, the relationship is characterized as a straight line with slope $\beta_1$ and y-intercept $\beta_0$.


In [ ]:
# Train a linear regression model using sklearn
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(data[['TV']], data[['sales']])

In [ ]:
# The coef_ and intercept_ attributes contain parameter values
print(model_lr.coef_) # beta_1
print(model_lr.intercept_) # beta_0

In [ ]:
# Plot the data points and the optimal regression line.
m = model_lr.coef_[0, 0]   # slope
b = model_lr.intercept_[0] # y-intercept

plt.plot(data['TV'], data['sales'], 'b.')
x_coordinates = np.array([0,300])
y_coordinates = x_coordinates * m + b
plt.plot(x_coordinates, y_coordinates, 'g-')

## Train A Simple Linear Regression Model

- For a given set of model parameters, we need a **cost function** (in some occasions also called **loss function**) that measures how well a given line fits the data.
- We also need a **training algorithm** that finds values of parameters so that the line fits the data well (usually "fitting the data well" means "minimizing the cost").

For linear regression:
- Cost function: 
    - Mean squared error (MSE)
    - Mean absolute error (MAE)
    - Other variants
- Training algorithm: 
    - Normal equation
    - Gradient descent

## Simple Linear Regression: Cost Function
A common choice of cost function is the **mean squared error (MSE) function**. It is defined as
$$\begin{align}
MSE(\beta_0, \beta_1) =& 
\frac{1}{N}\sum_{i=1}^N (y^{(i)} - f(x^{(i)}))^2 \\
=& \frac{1}{N}\sum_{i=1}^N\big(y^{(i)} - \beta_0 - \beta_1x^{(i)}\big)^2.
\end{align}$$

To better understand the MSE function, let's calculate the value $MSE(7.03, 0.04)$.

In [ ]:
# Example:
# Calculate the squared error of the model on the first record.

beta0 = 7.03
beta1 = 0.04

x1 = data.loc[1, 'TV'] # 230.1
y1 = data.loc[1, 'sales'] #22.1
print("x1, y1:", x1, y1)

# Calculate f(x1) = beta0 + beta1 * x1
f_x1 = beta0 + beta1 * x1

# Calculate the squared error (y1 - f(x1)) ** 2
squared_error = (y1 - f_x1) ** 2

# squared_error = (y1 - beta0 - beta1 * x1) ** 2

print("Squared error from Record 1:", squared_error)

In [ ]:
# Example:
# Calculate the squared error of the model on an arbitrary record.

beta0 = 7.03
beta1 = 0.04

i = 100  # index of the record
xi = data.loc[i, 'TV']
yi = data.loc[i, 'sales']
print("xi, yi:", xi, yi)

# Calculate f(xi)
f_xi = beta0 + beta1 * xi

# Calculate the squared error (yi - f(xi)) ** 2
squared_error = (yi - f_xi) ** 2

print("Squared error for Record %d:" % i, squared_error)

In [ ]:
# data[data.index == i]

In [ ]:
# Write a function that produces the squared error given beta0, beta1, data, and i
def get_squared_error(beta0, beta1, data, i):

    xi = data.loc[i, 'TV']
    yi = data.loc[i, 'sales']
    # print("xi, yi:", xi, yi)

    # Calculate f(xi)
    f_xi = beta0 + beta1 * xi

    # Calculate the squared error (yi - f(xi)) ** 2
    squared_error = (yi - f_xi) ** 2

    return squared_error

In [ ]:
get_squared_error(7.03, 0.04, data, 100)

In [ ]:
# Create a list that contains value of (y_1 - f(x_1))^2 for i=1,...,200.
list_errors = []
for ind in data.index:
    # call function `get_sqaured_error` with ind
    squared_error = get_squared_error(7.03, 0.04, data, ind)
    # append squared error to `list_errors`
    list_errors.append(squared_error)
    
print(list_errors)

In [ ]:
# Create list_errors using list comprehension

list_errors = [get_squared_error(7.03, 0.04, data, ind) for ind in data.index]
print(list_errors)

In [ ]:
# Calculate the MSE

MSE = sum(list_errors) / len(list_errors)

print("MSE:", MSE)

In [ ]:
# Write a function MSE(beta0, beta1, data) that returns the value of MSE with given beta0 and beta1.
def get_MSE(beta0, beta1, data):

    list_errors = [get_squared_error(beta0, beta1, data, ind) for ind in data.index]

    MSE = sum(list_errors) / len(list_errors)

    return MSE

Now, use function `get_MSE` to obtain the MSE for the following two sets of parameter values
- Case 1: $\beta_0 = 7.03, \beta_1 = 0.04$
- Case 2: $\beta_0 = 5, \beta_1 = 1$

Which one fits the data better?

In [ ]:
get_MSE(7.04, 0.04, data)

In [ ]:
get_MSE(5, 1, data)

Now that we have obtained the cost function, our next goal is to find the parameter values that minimizes the cost value.

In [ ]:
data

In fact, we can use `mean_squared_error` function from `sklearn.metrics` to calculate the MSE.

In [ ]:
from sklearn.metrics import mean_squared_error
beta0 = 5
beta1 = 1
y_pred = beta0 + beta1 * data['TV']
mean_squared_error(data['sales'], y_pred)


# Simple Linear Regression: Training Algorithm
To find the value of $\beta_0, \beta_1$ that minimizes the MSE cost function, there is a formula called the **normal equation** that gives the result directly:

$$\begin{pmatrix} \beta_0 \\ \beta_1 \end{pmatrix} = (\textbf{X}^T\cdot\textbf{X})^{-1}\cdot\textbf{X}^T\cdot\textbf{y}.$$

- $\textbf{X}$ is the matrix formed as 
$$\textbf{X} = \begin{pmatrix} 
1 & x^{(1)} \\
1 & x^{(2)} \\
\vdots & \vdots \\
1 & x^{(N)} \\
\end{pmatrix}.$$
- $\textbf{X}^T$ represents the **matrix transpose** of $\textbf{X}$.
- $\cdot$ represents **matrix multiplication**.
- $^{-1}$ represents **matrix inverse**.
- $\textbf{y}$ is the vector of target values
$$\textbf{y} = \begin{pmatrix} 
y^{(1)} \\
y^{(2)} \\
\vdots \\
y^{(N)} \\
\end{pmatrix}.$$

Let's apply the normal equation and find the best parameter values.

In [ ]:
# Construct X and y as numpy arrays
X = np.hstack([np.ones([len(data), 1]), data[['TV']].values])
# print(X)
y = data[['sales']].values
# print(y)

# X.T # matrix transpose
# A.dot(B) # matrix multiplication
# np.linalg.inv(X) # matrix inverse

beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
# # np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(beta)

In [ ]:
get_MSE(7.0325955, 0.04753664, data)

In [ ]:
# Plot the data points and the optimal regression line.
plt.plot(data['TV'], data['sales'], 'b.')
x_coordinates = np.array([0,300])
y_coordinates = x_coordinates * 0.047 + 7.03
plt.plot(x_coordinates, y_coordinates, 'g-')